In [54]:
%logstop
%logstart -rtq ~/.logs/pw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [4]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [5]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [6]:
import gzip
import simplejson as json

In [7]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [8]:
print(len(scripts))
scripts[:2]

382726


[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [9]:
print(len(practices))
practices[:2]

12020


[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [10]:
import numpy as np

def describe(key):
    list_values = []
    for dic in scripts:
        for k, v in dic.items():
            if k == key:
                list_values.append(v)

    total = sum(list_values)
    avg = np.mean(list_values)
    s = np.std(list_values)
    q25 = np.quantile(list_values, 0.25)
    med = np.median(list_values)
    q75 = np.quantile(list_values, 0.75)

    return (total, avg, s, q25, med, q75)

In [11]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [12]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [13]:
dic0 = scripts[0]
dic0['bnf_name']

'Co-Magaldrox_Susp 195mg/220mg/5ml S/F'

In [14]:
import numpy as np
list_all_bnf = []
for dic in scripts:
    list_all_bnf.append(dic['bnf_name'])
    
bnf_names = np.unique(list_all_bnf)
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [15]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    if script['bnf_name'] in bnf_names:
        groups[script['bnf_name']].append(script['items'])
print(groups)

{'3m Health Care_Cavilon Durable Barrier C': [3, 2, 7, 4, 1, 7, 2, 1, 4, 6, 5, 2, 5, 1, 4, 3, 3, 1, 3, 10, 1, 2, 2, 1, 1, 1, 1, 4, 1, 1, 4, 1, 2, 1, 1, 1, 2, 3, 3, 1, 3, 1, 5, 1, 1, 2, 8, 2, 1, 2, 1, 3, 1, 1, 4, 7, 4, 2, 4, 2, 2, 4, 19, 5, 4, 2, 1, 2, 7, 14, 1, 4, 8, 1, 1, 6, 3, 4, 5, 2, 1, 1, 16, 1, 9, 2, 1, 1, 1, 1, 1, 1, 5, 2, 1, 1, 4, 1, 4, 1, 1, 1, 1, 1, 1, 1, 6, 2, 12, 1, 13, 7, 4, 1, 1, 2, 1, 3, 5, 2, 1, 1, 3, 1, 4, 5, 4, 3, 1, 1, 2, 1, 19, 2, 13, 2, 1, 2, 1, 1, 7, 3, 19, 1, 2, 6, 4, 4, 4, 3, 1, 3, 1, 5, 1, 8, 5, 8, 1, 18, 6, 2, 7, 1, 6, 1, 1, 1, 4, 4, 3, 5, 14, 1, 2, 1, 1, 5, 1, 2, 2, 2, 1, 2, 1, 1, 6, 4, 8, 4, 4, 1, 1, 1, 3, 1, 2, 3, 3, 1, 2, 1, 2, 7, 2, 65, 10, 43, 3, 2, 20, 1, 6, 3, 16, 8, 1, 6, 3, 6, 1, 10, 5, 3, 1, 1, 4, 1, 1, 1, 1, 1, 2, 3, 2, 33, 9, 1, 1, 3, 2, 5, 1, 1, 4, 6, 1, 1, 1, 3, 1, 1, 3, 12, 33, 3, 1, 2, 3, 5, 6, 7, 6, 1, 6, 2, 36, 6, 1, 4, 4, 10, 2, 3, 2, 2, 1, 1, 11, 4, 30, 15, 7, 4, 2, 3, 3, 3, 14, 1, 39, 9, 37, 1, 6, 1, 4, 16, 10, 2, 23, 10, 23, 2, 4, 18, 3,

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [16]:
sum_up_groups = {name: [] for name in bnf_names}
for name in bnf_names:
    sum_up_groups[name] = sum(groups[name])

In [17]:
sum_up_groups

{'3m Health Care_Cavilon Durable Barrier C': 1998,
 '3m Health Care_Cavilon No Sting 1ml Barr': 247,
 '3m Health Care_Cavilon No Sting 3ml Barr': 39,
 '3m Health Care_Cavilon No Sting Barrier': 350,
 'A.S Saliva Orthana Spy 50ml (App)': 25,
 'A2A Spacer': 46,
 'A2A Spacer + Sml/Med Mask': 36,
 'AMI_Suportx Ostomy & A/Support Ladies Br': 2,
 'AMI_Suportx Ostomy/Hernia Easy Peel Supp': 9,
 'APO-go PFS_Inf 5mg/ml 10ml Pfs': 2,
 'Abasaglar KwikPen_100u/ml 3ml Pf Pen': 97,
 'Abasaglar_100u/ml 3ml Cart': 5,
 'Abidec_Dps': 1233,
 'Able Spacer': 276,
 'Able Spacer + Sml/Med Mask': 77,
 'Abtard_Tab 10mg M/R': 2,
 'Abtard_Tab 20mg M/R': 8,
 'Acamprosate Calc_Tab E/C 333mg': 388,
 'Acarbose_Tab 100mg': 6,
 'Acarbose_Tab 50mg': 51,
 'Accrete D3_Tab': 5993,
 'Aceclofenac_Tab 100mg': 5,
 'Acemetacin_Cap 60mg': 5,
 'Acenocoumarol_Tab 1mg': 39,
 'Acetazolamide_Cap 250mg M/R': 44,
 'Acetazolamide_Tab 250mg': 182,
 'Acetic Acid_Ear Spy 2% 5ml': 256,
 'Acetylcy_Cap 600mg': 2,
 'Acetylcy_Eye Dps 5%': 26,


In [18]:
# find the max item
key_list = list(sum_up_groups.keys())
val_list = list(sum_up_groups.values())
max_index = val_list.index(max(val_list))
max_item = [(key_list[max_index], val_list[max_index])]

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [19]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [20]:
scripts[0]

{'bnf_code': '0101010G0AAABAB',
 'items': 2,
 'practice': 'N81013',
 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
 'nic': 5.98,
 'act_cost': 5.56,
 'quantity': 1000}

In [21]:
def group_by_field(data, fields):
    names = {tuple(item[field] for field in fields) for item in data}
    groups = {name: [] for name in names}
    for item in data:
        name = tuple(item[field] for field in fields)
        groups[name].append(item)
    return groups

In [22]:
rst = group_by_field(scripts, ('bnf_name', 'practice'))

In [23]:
groups = group_by_field(scripts, ('bnf_name',))
test_max_item = [max ([(sum([member['items']
                           for member in group]), name[0])
                      for name, group in groups.items()])[::-1]]

In [24]:
test_max_item

[('Omeprazole_Cap E/C 20mg', 113826)]

In [25]:
max_item

[('Omeprazole_Cap E/C 20mg', 113826)]

In [26]:
assert test_max_item == max_item

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [27]:
print(scripts[0])
print()
print(practices[0])

{'bnf_code': '0101010G0AAABAB', 'items': 2, 'practice': 'N81013', 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F', 'nic': 5.98, 'act_cost': 5.56, 'quantity': 1000}

{'code': 'A81001', 'name': 'THE DENSHAM SURGERY', 'addr_1': 'THE HEALTH CENTRE', 'addr_2': 'LAWSON STREET', 'borough': 'STOCKTON ON TEES', 'village': 'CLEVELAND', 'post_code': 'TS18 1HU'}


In [28]:
practice_postal = {}
for practice in practices:
    code = practice['code']
    if practice['code'] not in practice_postal:
        practice_postal[practice['code']] = practice['post_code']
    else:
        practice_postal[practice['code']] = min(practice_postal[code], practice['post_code'])

In [29]:
practice_postal

{'A81001': 'TS18 1HU',
 'A81002': 'TS18 2AW',
 'A81003': 'TS25 1QU',
 'A81004': 'TS1 3BE',
 'A81005': 'TS14 7DJ',
 'A81006': 'TS18 2AT',
 'A81007': 'TS24 7PW',
 'A81008': 'TS6 6TD',
 'A81009': 'TS5 6HF',
 'A81011': 'TS24 7PW',
 'A81012': 'TS3 6AL',
 'A81013': 'TS12 2FF',
 'A81014': 'TS23 2LA',
 'A81015': 'TS10 1TZ',
 'A81016': 'TS1 3QY',
 'A81017': 'TS17 0EE',
 'A81018': 'TS10 4NW',
 'A81019': 'TS3 7RL',
 'A81020': 'TS4 3BU',
 'A81021': 'TS6 6TD',
 'A81022': 'TS12 2TG',
 'A81023': 'TS1 2NX',
 'A81025': 'TS18 1HU',
 'A81026': 'TS5 6HA',
 'A81027': 'TS15 9DD',
 'A81029': 'TS1 2NX',
 'A81030': 'TS1 3RY',
 'A81031': 'TS24 7PW',
 'A81032': 'TS14 7DJ',
 'A81033': 'TS3 6AL',
 'A81034': 'TS17 0EE',
 'A81035': 'TS1 3RX',
 'A81036': 'TS20 2UZ',
 'A81037': 'TS1 2NX',
 'A81038': 'TS3 6AL',
 'A81039': 'TS16 9EA',
 'A81040': 'TS23 2DG',
 'A81041': 'TS24 9DN',
 'A81042': 'TS6 9QG',
 'A81043': 'TS6 0HA',
 'A81044': 'TS25 1QU',
 'A81045': 'TS10 1SR',
 'A81046': 'TS18 1YE',
 'A81047': 'TS11 6BW',
 'A810

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [30]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [31]:
joined = scripts[:]
for script in joined:
    script['post_code'] = practice_postal[script['practice']]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [32]:
pc_grouped = list(group_by_field(joined, ('post_code',)).items())
items_by_post = [(pc[0], sum(group['items'] for group in groups)) for pc, groups in pc_grouped]

In [33]:
items_by_post = sorted(items_by_post)
items_by_post

[('B11 4BW', 20673),
 ('B18 7AL', 19001),
 ('B21 9RY', 29103),
 ('B23 6DJ', 24859),
 ('B70 7AW', 36531),
 ('BB11 2DL', 34356),
 ('BB2 1AX', 28254),
 ('BB3 1PY', 54514),
 ('BB4 5SL', 29388),
 ('BB7 2JG', 44585),
 ('BB8 0JZ', 54380),
 ('BB9 7SR', 38224),
 ('BD3 8QH', 21010),
 ('BH18 8EE', 39413),
 ('BH23 3AF', 32545),
 ('BL1 8TU', 26132),
 ('BL3 5HP', 27147),
 ('BL9 0NJ', 32062),
 ('BL9 0SN', 35275),
 ('CB9 8HF', 51337),
 ('CH1 4DS', 34915),
 ('CH65 6TG', 25090),
 ('CT11 8AD', 44358),
 ('CV1 4FS', 37210),
 ('CW1 3AW', 64104),
 ('CW5 5NX', 38797),
 ('CW7 1AT', 43164),
 ('DA1 2HA', 26075),
 ('DA11 8BZ', 24090),
 ('DN22 7XF', 43091),
 ('DN34 4GB', 48013),
 ('FY2 0JG', 69118),
 ('FY4 1TJ', 62886),
 ('FY5 2TZ', 44258),
 ('FY7 8GU', 34473),
 ('GL1 3PX', 38120),
 ('GL50 4DP', 74822),
 ('GU9 9QS', 32131),
 ('HA0 4UZ', 22755),
 ('HA3 7LT', 32113),
 ('HG1 5AR', 32684),
 ('HU7 4DW', 49107),
 ('KT14 6DH', 26758),
 ('KT6 6EZ', 38975),
 ('L31 0DJ', 32065),
 ('L36 7XY', 22965),
 ('L5 0QW', 24676),
 ('L

In [34]:
postal_totals = items_by_post[:100]

In [35]:
#postal_totals = [('B11 4BW', 20673)] * 100

grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [36]:
name_grouped = group_by_field(joined, ('post_code', 'bnf_name')).items()

In [38]:
total_items_by_bnf_post= {name: sum(group['items'] for group in groups) for name, groups in name_grouped}
assert len(total_items_by_bnf_post) == 141196

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [40]:
total_items = [{'post_code': tup[0], 'bnf_name': tup[1], 'total': tot} for tup, tot in list(total_items_by_bnf_post.items())]
total_items_by_post = group_by_field(total_items, ('post_code',))
assert len(total_items_by_post) == 118

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [42]:
max_item_by_post = [sorted(group, key = lambda mem: (-mem['total'], mem['bnf_name']))[0] for group in total_items_by_post.values()]

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [46]:
rst = [(item['post_code'], item['bnf_name'], item['total']/dict(items_by_post)[item['post_code']]) for item in max_item_by_post]

In [47]:
items_by_region = sorted(rst)[:100]

In [ ]:
#items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [48]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*